# https://frhyme.github.io/python/python_korean_englished/

In [11]:
# https://frhyme.github.io/python/python_korean_englished/
# 복붙

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

def korean_to_be_englished(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst
    
korean_to_be_englished("이승훈a")

def korean_word_to_initials(korean_word):
    """
    한글을 입력받아서 한글 초성에 따라서 이니셜로 변환해줍니다.
    한국 성의 경우 조금 다르게 변환되는데 '박' ==> 'Park'인 부분은 반영하지 않음 
    """
    w_to_k = {'ㄱ':'K', 'ㄲ':'G', 'ㄴ':'N', 'ㄷ':'D', 'ㄸ':'D', 'ㄹ':'R', 'ㅁ':'M', 'ㅂ':'B', 
              'ㅃ':'B', 'ㅅ':'S', 'ㅈ':'J', 'ㅉ':'J', 'ㅊ':'C', 'ㅌ':'T', 'ㅍ':'P', 'ㅎ':'H'}
    r_lst = []
    for i, w in enumerate(korean_to_be_englished(korean_word)):
        if w[0] in w_to_k.keys():
            r_lst.append( w_to_k[w[0]] )
        else:
            if w[1] in ['ㅑ', 'ㅕ', 'ㅛ', 'ㅠ', 'ㅖ']: 
                r_lst.append('Y')
            elif w[1] in ['ㅝ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅜ', 'ㅞ', 'ㅟ']:
                r_lst.append('W')
            elif w[1] in ['ㅔ', 'ㅡ', 'ㅢ']:
                r_lst.append('E')
            elif w[1] in ['ㅏ', 'ㅐ']:
                r_lst.append('A')
            elif w[1] in ['ㅓ']:
                r_lst.append('U')
            elif w[1] in ['ㅗ']:
                r_lst.append('O')
            elif w[1] in ['ㅣ']:
                if i==0: 
                    r_lst.append('L')
                else:
                    r_lst.append('I')
            else:
                return 'not applicable'
    return "".join(r_lst)

### test case
test_cases = ['이승훈', '전지현', '하정우', '정우성', '박상원', '정유미', '김연우', '윤종신']
for case in test_cases:
    print("{} ==> {}".format(case, korean_word_to_initials(case)))


이승훈 ==> LSH
전지현 ==> JJH
하정우 ==> HJW
정우성 ==> JWS
박상원 ==> BSW
정유미 ==> JYM
김연우 ==> KYW
윤종신 ==> YJS


# https://github.com/neotune/python-korean-handler/blob/master/korean_handler.py

In [1]:
# -*- coding: utf-8 -*-
import re
import sys

# 위주소에서 글에서 참고했다는 깃헙의 코드를 복붙한것이 밑의 것임.
"""
    초성 중성 종성 분리 하기
	유니코드 한글은 0xAC00 으로부터
	초성 19개, 중성21개, 종성28개로 이루어지고
	이들을 조합한 11,172개의 문자를 갖는다.
	한글코드의 값 = ((초성 * 21) + 중성) * 28 + 종성 + 0xAC00
	(0xAC00은 'ㄱ'의 코드값)
	따라서 다음과 같은 계산 식이 구해진다.
	유니코드 한글 문자 코드 값이 X일 때,
	초성 = ((X - 0xAC00) / 28) / 21
	중성 = ((X - 0xAC00) / 28) % 21
	종성 = (X - 0xAC00) % 28
	이 때 초성, 중성, 종성의 값은 각 소리 글자의 코드값이 아니라
	이들이 각각 몇 번째 문자인가를 나타내기 때문에 다음과 같이 다시 처리한다.
	초성문자코드 = 초성 + 0x1100 //('ㄱ')
	중성문자코드 = 중성 + 0x1161 // ('ㅏ')
	종성문자코드 = 종성 + 0x11A8 - 1 // (종성이 없는 경우가 있으므로 1을 뺌)
"""
# 유니코드 한글 시작 : 44032, 끝 : 55199
BASE_CODE, CHOSUNG, JUNGSUNG = 44032, 588, 28

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

def convert(test_keyword):
    split_keyword_list = list(test_keyword)
    #print(split_keyword_list)

    result = list()
    for keyword in split_keyword_list:
        # 한글 여부 check 후 분리
        if re.match('.*[ㄱ-ㅎㅏ-ㅣ가-힣]+.*', keyword) is not None:
            char_code = ord(keyword) - BASE_CODE
            char1 = int(char_code / CHOSUNG)
            result.append(CHOSUNG_LIST[char1])
            #print('초성 : {}'.format(CHOSUNG_LIST[char1]))
            char2 = int((char_code - (CHOSUNG * char1)) / JUNGSUNG)
            result.append(JUNGSUNG_LIST[char2])
            #print('중성 : {}'.format(JUNGSUNG_LIST[char2]))
            char3 = int((char_code - (CHOSUNG * char1) - (JUNGSUNG * char2)))
            if char3==0:
                result.append('#')
            else:
                result.append(JONGSUNG_LIST[char3])
            #print('종성 : {}'.format(JONGSUNG_LIST[char3]))
        else:
            result.append(keyword)
    # result
    print("".join(result))

if __name__ == '__main__':
    
    if len(sys.argv) > 1:
        inputfile = open(sys.argv[1], 'r')
        for line in inputfile.readlines():
            convert(line)
    else:
        test_keyword = input("input your text:")
        convert(test_keyword)

# my try

In [4]:
ord('ㄱ')

12593

In [5]:
ord('힣')

55203

In [6]:
for i in range(12590,55205):
    print(chr(i))

ㄮ
ㄯ
㄰
ㄱ
ㄲ
ㄳ
ㄴ
ㄵ
ㄶ
ㄷ
ㄸ
ㄹ
ㄺ
ㄻ
ㄼ
ㄽ
ㄾ
ㄿ
ㅀ
ㅁ
ㅂ
ㅃ
ㅄ
ㅅ
ㅆ
ㅇ
ㅈ
ㅉ
ㅊ
ㅋ
ㅌ
ㅍ
ㅎ
ㅏ
ㅐ
ㅑ
ㅒ
ㅓ
ㅔ
ㅕ
ㅖ
ㅗ
ㅘ
ㅙ
ㅚ
ㅛ
ㅜ
ㅝ
ㅞ
ㅟ
ㅠ
ㅡ
ㅢ
ㅣ
ㅤ
ㅥ
ㅦ
ㅧ
ㅨ
ㅩ
ㅪ
ㅫ
ㅬ
ㅭ
ㅮ
ㅯ
ㅰ
ㅱ
ㅲ
ㅳ
ㅴ
ㅵ
ㅶ
ㅷ
ㅸ
ㅹ
ㅺ
ㅻ
ㅼ
ㅽ
ㅾ
ㅿ
ㆀ
ㆁ
ㆂ
ㆃ
ㆄ
ㆅ
ㆆ
ㆇ
ㆈ
ㆉ
ㆊ
ㆋ
ㆌ
ㆍ
ㆎ
㆏
㆐
㆑
㆒
㆓
㆔
㆕
㆖
㆗
㆘
㆙
㆚
㆛
㆜
㆝
㆞
㆟
ㆠ
ㆡ
ㆢ
ㆣ
ㆤ
ㆥ
ㆦ
ㆧ
ㆨ
ㆩ
ㆪ
ㆫ
ㆬ
ㆭ
ㆮ
ㆯ
ㆰ
ㆱ
ㆲ
ㆳ
ㆴ
ㆵ
ㆶ
ㆷ
ㆸ
ㆹ
ㆺ
ㆻ
ㆼ
ㆽ
ㆾ
ㆿ
㇀
㇁
㇂
㇃
㇄
㇅
㇆
㇇
㇈
㇉
㇊
㇋
㇌
㇍
㇎
㇏
㇐
㇑
㇒
㇓
㇔
㇕
㇖
㇗
㇘
㇙
㇚
㇛
㇜
㇝
㇞
㇟
㇠
㇡
㇢
㇣
㇤
㇥
㇦
㇧
㇨
㇩
㇪
㇫
㇬
㇭
㇮
㇯
ㇰ
ㇱ
ㇲ
ㇳ
ㇴ
ㇵ
ㇶ
ㇷ
ㇸ
ㇹ
ㇺ
ㇻ
ㇼ
ㇽ
ㇾ
ㇿ
㈀
㈁
㈂
㈃
㈄
㈅
㈆
㈇
㈈
㈉
㈊
㈋
㈌
㈍
㈎
㈏
㈐
㈑
㈒
㈓
㈔
㈕
㈖
㈗
㈘
㈙
㈚
㈛
㈜
㈝
㈞
㈟
㈠
㈡
㈢
㈣
㈤
㈥
㈦
㈧
㈨
㈩
㈪
㈫
㈬
㈭
㈮
㈯
㈰
㈱
㈲
㈳
㈴
㈵
㈶
㈷
㈸
㈹
㈺
㈻
㈼
㈽
㈾
㈿
㉀
㉁
㉂
㉃
㉄
㉅
㉆
㉇
㉈
㉉
㉊
㉋
㉌
㉍
㉎
㉏
㉐
㉑
㉒
㉓
㉔
㉕
㉖
㉗
㉘
㉙
㉚
㉛
㉜
㉝
㉞
㉟
㉠
㉡
㉢
㉣
㉤
㉥
㉦
㉧
㉨
㉩
㉪
㉫
㉬
㉭
㉮
㉯
㉰
㉱
㉲
㉳
㉴
㉵
㉶
㉷
㉸
㉹
㉺
㉻
㉼
㉽
㉾
㉿
㊀
㊁
㊂
㊃
㊄
㊅
㊆
㊇
㊈
㊉
㊊
㊋
㊌
㊍
㊎
㊏
㊐
㊑
㊒
㊓
㊔
㊕
㊖
㊗
㊘
㊙
㊚
㊛
㊜
㊝
㊞
㊟
㊠
㊡
㊢
㊣
㊤
㊥
㊦
㊧
㊨
㊩
㊪
㊫
㊬
㊭
㊮
㊯
㊰
㊱
㊲
㊳
㊴
㊵
㊶
㊷
㊸
㊹
㊺
㊻
㊼
㊽
㊾
㊿
㋀
㋁
㋂
㋃
㋄
㋅
㋆
㋇
㋈
㋉
㋊
㋋
㋌
㋍
㋎
㋏
㋐
㋑
㋒
㋓
㋔
㋕
㋖
㋗
㋘
㋙
㋚
㋛
㋜
㋝
㋞
㋟
㋠
㋡
㋢
㋣
㋤
㋥
㋦
㋧
㋨
㋩
㋪
㋫
㋬
㋭
㋮
㋯
㋰
㋱
㋲
㋳
㋴
㋵
㋶
㋷
㋸
㋹
㋺
㋻
㋼
㋽
㋾
㋿
㌀
㌁
㌂
㌃
㌄
㌅
㌆
㌇
㌈
㌉
㌊
㌋
㌌
㌍
㌎
㌏
㌐
㌑
㌒
㌓
㌔
㌕
㌖
㌗
㌘
㌙
㌚
㌛
㌜
㌝
㌞
㌟
㌠
㌡


In [8]:
[print(a) for a in range(10)]

0
1
2
3
4
5
6
7
8
9


[None, None, None, None, None, None, None, None, None, None]

In [9]:
unicode_test_list=[chr(i) for i in range(12590,55205)]

In [10]:
unicode_test_list

['ㄮ',
 'ㄯ',
 '\u3130',
 'ㄱ',
 'ㄲ',
 'ㄳ',
 'ㄴ',
 'ㄵ',
 'ㄶ',
 'ㄷ',
 'ㄸ',
 'ㄹ',
 'ㄺ',
 'ㄻ',
 'ㄼ',
 'ㄽ',
 'ㄾ',
 'ㄿ',
 'ㅀ',
 'ㅁ',
 'ㅂ',
 'ㅃ',
 'ㅄ',
 'ㅅ',
 'ㅆ',
 'ㅇ',
 'ㅈ',
 'ㅉ',
 'ㅊ',
 'ㅋ',
 'ㅌ',
 'ㅍ',
 'ㅎ',
 'ㅏ',
 'ㅐ',
 'ㅑ',
 'ㅒ',
 'ㅓ',
 'ㅔ',
 'ㅕ',
 'ㅖ',
 'ㅗ',
 'ㅘ',
 'ㅙ',
 'ㅚ',
 'ㅛ',
 'ㅜ',
 'ㅝ',
 'ㅞ',
 'ㅟ',
 'ㅠ',
 'ㅡ',
 'ㅢ',
 'ㅣ',
 'ㅤ',
 'ㅥ',
 'ㅦ',
 'ㅧ',
 'ㅨ',
 'ㅩ',
 'ㅪ',
 'ㅫ',
 'ㅬ',
 'ㅭ',
 'ㅮ',
 'ㅯ',
 'ㅰ',
 'ㅱ',
 'ㅲ',
 'ㅳ',
 'ㅴ',
 'ㅵ',
 'ㅶ',
 'ㅷ',
 'ㅸ',
 'ㅹ',
 'ㅺ',
 'ㅻ',
 'ㅼ',
 'ㅽ',
 'ㅾ',
 'ㅿ',
 'ㆀ',
 'ㆁ',
 'ㆂ',
 'ㆃ',
 'ㆄ',
 'ㆅ',
 'ㆆ',
 'ㆇ',
 'ㆈ',
 'ㆉ',
 'ㆊ',
 'ㆋ',
 'ㆌ',
 'ㆍ',
 'ㆎ',
 '\u318f',
 '㆐',
 '㆑',
 '㆒',
 '㆓',
 '㆔',
 '㆕',
 '㆖',
 '㆗',
 '㆘',
 '㆙',
 '㆚',
 '㆛',
 '㆜',
 '㆝',
 '㆞',
 '㆟',
 'ㆠ',
 'ㆡ',
 'ㆢ',
 'ㆣ',
 'ㆤ',
 'ㆥ',
 'ㆦ',
 'ㆧ',
 'ㆨ',
 'ㆩ',
 'ㆪ',
 'ㆫ',
 'ㆬ',
 'ㆭ',
 'ㆮ',
 'ㆯ',
 'ㆰ',
 'ㆱ',
 'ㆲ',
 'ㆳ',
 'ㆴ',
 'ㆵ',
 'ㆶ',
 'ㆷ',
 'ㆸ',
 'ㆹ',
 'ㆺ',
 '\u31bb',
 '\u31bc',
 '\u31bd',
 '\u31be',
 '\u31bf',
 '㇀',
 '㇁',
 '㇂',
 '㇃',
 '㇄',
 '㇅',
 '㇆',
 '㇇',
 '㇈',
 '㇉',
 '㇊',
 '㇋',
 '㇌',
 '㇍',
 '㇎',

In [14]:
0xAC00

44032

In [15]:
ord('가')

44032

In [17]:
ord('ㄱ')

12593

In [19]:
ord('ㅎ')

12622

In [20]:
ord('ㅏ')

12623

In [21]:
ord('ㅣ')

12643

In [22]:
ord('가')

44032

In [23]:
ord('힣')

55203

In [24]:
print(chr(12592),chr(12644),chr(44031),chr(55204))

㄰ ㅤ ꯿ 힤


In [27]:
for i in range(12623, 12646):
    print(chr(i))

ㅏ
ㅐ
ㅑ
ㅒ
ㅓ
ㅔ
ㅕ
ㅖ
ㅗ
ㅘ
ㅙ
ㅚ
ㅛ
ㅜ
ㅝ
ㅞ
ㅟ
ㅠ
ㅡ
ㅢ
ㅣ
ㅤ
ㅥ


In [29]:
for i in range(12623, 12644):
    print(chr(i))

ㅏ
ㅐ
ㅑ
ㅒ
ㅓ
ㅔ
ㅕ
ㅖ
ㅗ
ㅘ
ㅙ
ㅚ
ㅛ
ㅜ
ㅝ
ㅞ
ㅟ
ㅠ
ㅡ
ㅢ
ㅣ


 연합뉴스TV
[30초뉴스] 홍남기 인용한 '지지지지(知止止止)' 뜻이 뭐길래…해석 분분 기사입력 2021.02.04. 오후 3:23 https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=422&aid=0000469833

In [36]:
t = '''홍남기 부총리 겸 기획재정부 장관이 전국민 4차 재난지원금 지급에 반대 의사를 표명하면서 언급한 '지지지지(知止止止)' 표현이 회자되고 있습니다. 이 사자성어는 '도덕경'에 등장하는 표현으로 '그침을 알아 그칠 곳에서 그친다'는 뜻입니다. 홍 부총리가 왜 이 말을 인용했는지를 두고 해석이 분분합니다.

▶ 네이버에서 연합뉴스TV를 구독하세요
▶ 연합뉴스TV 생방송 만나보기
▶ 균형있는 뉴스, 연합뉴스TV 앱 다운받기
'''

In [37]:
t

"홍남기 부총리 겸 기획재정부 장관이 전국민 4차 재난지원금 지급에 반대 의사를 표명하면서 언급한 '지지지지(知止止止)' 표현이 회자되고 있습니다. 이 사자성어는 '도덕경'에 등장하는 표현으로 '그침을 알아 그칠 곳에서 그친다'는 뜻입니다. 홍 부총리가 왜 이 말을 인용했는지를 두고 해석이 분분합니다.\n\n▶ 네이버에서 연합뉴스TV를 구독하세요\n▶ 연합뉴스TV 생방송 만나보기\n▶ 균형있는 뉴스, 연합뉴스TV 앱 다운받기\n"

In [49]:
t2 = t.replace('\n','').split(' ')

In [50]:
t2

['홍남기',
 '부총리',
 '겸',
 '기획재정부',
 '장관이',
 '전국민',
 '4차',
 '재난지원금',
 '지급에',
 '반대',
 '의사를',
 '표명하면서',
 '언급한',
 "'지지지지(知止止止)'",
 '표현이',
 '회자되고',
 '있습니다.',
 '이',
 '사자성어는',
 "'도덕경'에",
 '등장하는',
 '표현으로',
 "'그침을",
 '알아',
 '그칠',
 '곳에서',
 "그친다'는",
 '뜻입니다.',
 '홍',
 '부총리가',
 '왜',
 '이',
 '말을',
 '인용했는지를',
 '두고',
 '해석이',
 '분분합니다.▶',
 '네이버에서',
 '연합뉴스TV를',
 '구독하세요▶',
 '연합뉴스TV',
 '생방송',
 '만나보기▶',
 '균형있는',
 '뉴스,',
 '연합뉴스TV',
 '앱',
 '다운받기']

 연합뉴스
홍남기 '지지지지'…직 걸고 전국민 지원금 반대 관철할까
기사입력 2021.02.03. 오후 12:21 최종수정 2021.02.03. 오후 1:47
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012181341

In [51]:
t3 = ''''그침을 안다는 知止' 표현, 거취 고민으로 해석돼
"지원금·추경 관련 이견 있어 절제된 표현으로 전달한 것" 진화


본회의 참석한 경제부총리.. '예산고민?'
(서울=연합뉴스) 안정원 기자 = 홍남기 경제부총리 겸 기획재정부 장관이 3일 오전 국회 본회의에서 열린 국민의힘 주호영 원내대표 교섭단체 대표연설에 참석, 잠시 눈을 감고 있다. jeong@yna.co.kr


(세종=연합뉴스) 박용주 기자 = 홍남기 부총리 겸 기획재정부 장관이 여당이 제시한 4차 재난지원금 지급 범위에 반대 의사를 표명하면서 쓴 '지지지지(知止止止)'란 표현이 화제가 되고 있다.

전 국민 재난지원금과 코로나19 피해계층에 대한 선별적 재난지원금을 동시에 추진하겠다는 여당의 방침을 저지하지 못할 경우 부총리 직에서 물러나겠다는 의미로 해석될 여지가 있는 까닭이다.

홍 부총리는 2일 이낙연 더불어민주당 대표의 교섭단체 연설 직후 자신의 페이스북에 올린 글에서 "추가적 재난지원금 지원이 불가피하다고 하더라도 전 국민 보편지원과 선별지원을 한꺼번에 모두 하겠다는 것은 정부로서는 받아들이기 어렵다"고 이 대표가 언급한 보편·선별 지원 병행 추진에 완곡한 표현으로 반대 의견을 표명했다.

그는 글의 말미에 "최선을 다한 사람은 결과에 연연하지 말고 담백하게 나아간다는 말이 있다. 그렇게 의연하고 담백하게 나아가기를 바란다"면서 "저부터 늘 가슴에 지지지지(知止止止)의 심정을 담고 하루하루 뚜벅뚜벅 걸어왔고 또 걸어갈 것"이라고 썼다.

이어 "우리 기재부 직원들의 뛰어난 역량과 고귀한 열정, 그리고 책임감 있는 사명감과 사투 의지를 믿고 응원한다"는 말로 끝을 맺었다.

지지지지는 도덕경에 나오는 표현으로 '그침을 알아 그칠 곳에서 그친다'는 표현이다. 본인의 거취를 깊이 있게 고민한다는 의미로 해석될 수 있다.

'기재부 직원의 사투 의지를 믿고 응원한다'는 표현도 의미심장하다는 얘기가 나온다.

주호영 원내대표 연설 듣는 홍남기 경제부총리
(서울=연합뉴스) 하사헌 기자 = 홍남기 부총리 겸 기획재정부 장관이 3일 오전 국회 본회의장에서 국민의힘 주호영 원내대표의 교섭단체 대표연설을 듣고 있다. toadboy@yna.co.kr


홍 부총리는 1·2·3차 재난지원금 지급과 추가경정예산 편성, 대주주 양도소득세 부과 기준 등을 두고 여당과 충돌하다 결국 물러서, '홍백기', '홍두사미'란 조롱 섞인 별명을 얻기도 했다. 그래서 이번에도 결국 의지를 꺾을 것이란 전망이 적지 않다.

홍 부총리는 지난해 11월에도 대주주 양도세 부과 기준을 놓고 문재인 대통령에게 사직서를 제출했으나 반려된 바 있다.

그는 대주주 기준을 기존 10억원에서 3억원으로 낮추는 방안을 예정대로 시행하려 했으나, 여당의 반대로 관철하지 못하자 누군가 책임을 져야 한다며 사의를 표명했다.

홍 부총리가 이번에 반대 입장을 표명한 수위가 예사롭지 않아 보인다는 얘기가 기재부 내에서 나온다. 직을 건 거 아니냐는 관측도 나온다.

하지만 홍 부총리는 이날에는 다소 톤을 낮췄다.

국회에서 기자들을 만난 자리에서 홍 부총리는 "어제 이낙연 대표의 연설은 공직 생활을 하면서 가장 격조 있고 정책 콘텐츠가 탄탄한 대표연설이었다"고 추켜세웠다. 또 "어제 페이스북 글은 재난지원금과 추경과 관련한 이견 사항이 확정된 것으로 전달될까 봐 재정 당국 입장을 절제된 표현으로 말씀드린 것"이었다고 설명했다.

홍 부총리는 "제가 드리고자 하는 내용은 어제 SNS에 잘 표현했다고 생각한다. 더는 답변하지 않겠다"며 자리를 떠났다.

speed@yna.co.kr

▶네이버에서도 뉴스는 연합뉴스[구독 클릭]
▶[팩트체크]담뱃값 싸서 흡연율 높다?·올리면 덜 피워?
▶제보하기

<저작권자(c) 연합뉴스(https://www.yna.co.kr/), 무단 전재-재배포 금지> '''

In [52]:
t4 = t3.replace('\n','').split(' ')

In [53]:
t4

["'그침을",
 '안다는',
 "知止'",
 '표현,',
 '거취',
 '고민으로',
 '해석돼"지원금·추경',
 '관련',
 '이견',
 '있어',
 '절제된',
 '표현으로',
 '전달한',
 '것"',
 '진화본회의',
 '참석한',
 '경제부총리..',
 "'예산고민?'(서울=연합뉴스)",
 '안정원',
 '기자',
 '=',
 '홍남기',
 '경제부총리',
 '겸',
 '기획재정부',
 '장관이',
 '3일',
 '오전',
 '국회',
 '본회의에서',
 '열린',
 '국민의힘',
 '주호영',
 '원내대표',
 '교섭단체',
 '대표연설에',
 '참석,',
 '잠시',
 '눈을',
 '감고',
 '있다.',
 'jeong@yna.co.kr(세종=연합뉴스)',
 '박용주',
 '기자',
 '=',
 '홍남기',
 '부총리',
 '겸',
 '기획재정부',
 '장관이',
 '여당이',
 '제시한',
 '4차',
 '재난지원금',
 '지급',
 '범위에',
 '반대',
 '의사를',
 '표명하면서',
 '쓴',
 "'지지지지(知止止止)'란",
 '표현이',
 '화제가',
 '되고',
 '있다.전',
 '국민',
 '재난지원금과',
 '코로나19',
 '피해계층에',
 '대한',
 '선별적',
 '재난지원금을',
 '동시에',
 '추진하겠다는',
 '여당의',
 '방침을',
 '저지하지',
 '못할',
 '경우',
 '부총리',
 '직에서',
 '물러나겠다는',
 '의미로',
 '해석될',
 '여지가',
 '있는',
 '까닭이다.홍',
 '부총리는',
 '2일',
 '이낙연',
 '더불어민주당',
 '대표의',
 '교섭단체',
 '연설',
 '직후',
 '자신의',
 '페이스북에',
 '올린',
 '글에서',
 '"추가적',
 '재난지원금',
 '지원이',
 '불가피하다고',
 '하더라도',
 '전',
 '국민',
 '보편지원과',
 '선별지원을',
 '한꺼번에',
 '모두',
 '하겠다는',
 '것은',
 '정부로서는',
 '받아들이기',
 '어렵다"고'

In [54]:
'금' in t4

False

In [55]:
'무' in t4

False

In [56]:
'무단' in t4

True

In [57]:
match1 = []
for i in t2:
    for j in t4:
        

SyntaxError: unexpected EOF while parsing (<ipython-input-57-2cbbf913c81c>, line 4)

In [58]:
ord(t4)

TypeError: ord() expected string of length 1, but list found

In [59]:
ord(t3)

TypeError: ord() expected a character, but string of length 1873 found

In [60]:
ord(t4[0])

TypeError: ord() expected a character, but string of length 4 found

In [61]:
ord(t4[0][0])

39

In [62]:
chr(39)

"'"

good reference for korean NLP

https://konlpy.org/ko/v0.4.4/

https://joyfuls.tistory.com/67?category=731429

korean NLP is to difficult to me...

https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_srt&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so%3Add%2Cp%3Aall%2Ca%3Aall&mynews=0&refresh_start=0&related=0

https://search.naver.com/search.naver?where=news&query=sk%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&sm=tab_srt&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so%3Add%2Cp%3Aall%2Ca%3Aall&mynews=0&refresh_start=0&related=0

lets just use numbers.

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

#from openpyxl import load_workbook
from openpyxl import Workbook

from datetime import date
import time

import sys

import os

url='https:naver.com'
res=urlopen(url)
soup=BeautifulSoup(res,'lxml')
#soup=BeautifulSoup(res,'html.parser')
print(soup)

URLError: <urlopen error no host given>